In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
from keras import backend as K
from keras.preprocessing.image import load_img, save_img, img_to_array
import matplotlib.pyplot as plt
from keras.applications import vgg19
from keras.models import Model
#from keras import optimizers
from scipy.optimize import fmin_l_bfgs_b
import os
import pandas as pd
import numpy as np
import imageio
from skimage.color import rgb2gray
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from PIL import Image, ImageFile
import urllib.request
import requests
from io import BytesIO
from random import choices

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Activation
from keras.optimizers import SGD

import tensorflow as tf
from tensorflow.keras import Model, layers
from sklearn.model_selection import StratifiedKFold
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D


In [52]:
path = "/Users/qianzhuang/Academic/Upenn/Capstone/model/image-classification/images/images/"
files = []

In [53]:
for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files.append(os.path.join(r, file))

In [55]:
df = pd.DataFrame(files)
df = df.rename({0:"path"},axis = 1)

In [59]:
def getLabel(string):
    return string.split('/')[-2]

In [57]:
df['labels'] = df.path.apply(getLabel)

In [58]:
df.head()

,path,labels
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure


In [99]:
df.path[0]

'/Users/qianzhuang/Academic/Upenn/Capstone/model/224/architecure/63.jpg'

In [46]:
# create a function to create a RGB pic
def creatRGB(array):
    array = imageio.imread(array)
    pixels = np.asarray(array)
    imge = pixels.astype('uint8')
    return imge

In [ ]:
df.head()

In [19]:
test_arr = df.RGB[0]

In [37]:
label_list = df.labels.unique()

In [61]:
from PIL import Image
import os

img_path = '/Users/qianzhuang/Academic/Upenn/Capstone/model/image-classification/images/images'
resize_path_head = '/Users/qianzhuang/Academic/Upenn/Capstone/model/64/'


In [62]:
for i in range(0,len(df)):
    im = Image.open(os.path.join(df.path[i]))
    out = im.resize((64, 64))
    resize_path = resize_path_head+df.labels[i]
    if not os.path.exists(resize_path):
        os.makedirs(resize_path)
    try:
        out.save(resize_path+"/"+str(i)+".jpg")
    except:
        os.remove(resize_path+"/"+str(i)+".jpg")

/Users/qianzhuang/anaconda/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:804: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [81]:
i

9935

In [56]:
df.path[i]

'/Users/qianzhuang/Academic/Upenn/Capstone/model/image-classification/images/images/art and culture/637 (2).jpg'

In [60]:
path = "/Users/qianzhuang/Academic/Upenn/Capstone/model/64/"
files_2 = []

for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files_2.append(os.path.join(r, file))

df = pd.DataFrame(files_2)
df = df.rename({0:"path"},axis = 1)
df['labels'] = df.path.apply(getLabel)
df.head()

,path,labels
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure


In [61]:
df['RGB'] = df.path.apply(creatRGB)

In [62]:
df.head()

,path,labels,RGB
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[59, 124, 180], [59, 124, 180], [59, 123, 18..."
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[6, 3, 0], [8, 5, 0], [10, 7, 2], [9, 8, 3],..."
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[173, 144, 100], [117, 88, 48], [75, 45, 9],..."
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[65, 72, 78], [61, 68, 76], [60, 67, 75], [6..."


In [63]:
list_label = df.labels.unique()
list_label = list(list_label)

In [64]:
df.RGB[0].shape

(64, 64, 3)

In [65]:
#drop the row not 224,224,3
drop_list = []
for i in range(0,len(df)):
    if(df.RGB[i].shape != (64,64,3)):
        drop_list.append(i)
df = df.drop(drop_list)

In [66]:
df.head()

,path,labels,RGB
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[59, 124, 180], [59, 124, 180], [59, 123, 18..."
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[6, 3, 0], [8, 5, 0], [10, 7, 2], [9, 8, 3],..."
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[173, 144, 100], [117, 88, 48], [75, 45, 9],..."
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[65, 72, 78], [61, 68, 76], [60, 67, 75], [6..."


In [67]:
df_ar = df.loc[df.labels == list_label[0]]
df_2 = df.loc[df.labels == list_label[1]]
df_3 = df.loc[df.labels == list_label[2]]
df_4 = df.loc[df.labels == list_label[3]]

df_ar = df_ar.sample(n=2000)
df_2 = df_2.sample(n=2000)
df_3 = df_3.sample(n=2000)
df_4 = df_4.sample(n=2000)

df_2000 = df_ar.append(df_2).append(df_3).append(df_4)

In [68]:
df_2000.head()

,path,labels,RGB
1310,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[117, 109, 96], [97, 89, 76], [28, 19, 4], [..."
3954,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[165, 121, 84], [131, 83, 47], [177, 123, 89..."
419,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[139, 137, 142], [181, 179, 184], [114, 112,..."
7642,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[198, 190, 171], [180, 172, 153], [190, 182,..."
8489,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[250, 249, 254], [242, 241, 246], [253, 254,..."


In [69]:
len(df_2000)

8000

In [18]:
df['RGB_one_dim'] = df.RGB.apply(changeDim)

In [17]:
def changeDim(array):
    return array.ravel()

In [70]:
def changeNum(name):
    return list_label.index(name)

In [71]:
df_2000['num'] = df_2000.labels.apply(changeNum)

In [72]:
df_2000.head()

,path,labels,RGB,num
1310,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[117, 109, 96], [97, 89, 76], [28, 19, 4], [...",0
3954,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[165, 121, 84], [131, 83, 47], [177, 123, 89...",0
419,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[139, 137, 142], [181, 179, 184], [114, 112,...",0
7642,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[198, 190, 171], [180, 172, 153], [190, 182,...",0
8489,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,architecure,"[[[250, 249, 254], [242, 241, 246], [253, 254,...",0


In [83]:
# set the batch size, class number and epochs
batch_size = 128
num_classes = 4
epochs = 15

In [74]:
# normalize the data by divide 255
X = np.array(df_2000.RGB.to_list())/255
y = np.array(df_2000.num.to_list())

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)



In [76]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [77]:
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
import ssl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from keras.applications import resnet50
from keras import optimizers

In [79]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (64,64,3)
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = input_shape))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = "softmax"))

In [80]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 32)        2432      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 16, 16, 64)       

In [81]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [84]:
# fit the training data and using testing data to valiadte
model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(X_test, y_test))

Train on 4800 samples, validate on 3200 samples
Epoch 1/15
4800/4800 [==============================] - 67s 14ms/step - loss: 1.3145 - acc: 0.3667 - val_loss: 1.2352 - val_acc: 0.4619
Epoch 2/15
4800/4800 [==============================] - 67s 14ms/step - loss: 1.2068 - acc: 0.4619 - val_loss: 1.1786 - val_acc: 0.4772
Epoch 3/15
4800/4800 [==============================] - 67s 14ms/step - loss: 1.1562 - acc: 0.4858 - val_loss: 1.1246 - val_acc: 0.5228
Epoch 4/15
4800/4800 [==============================] - 67s 14ms/step - loss: 1.1114 - acc: 0.5252 - val_loss: 1.1380 - val_acc: 0.5100
Epoch 5/15
4800/4800 [==============================] - 79s 16ms/step - loss: 1.0858 - acc: 0.5323 - val_loss: 1.1583 - val_acc: 0.4659
Epoch 6/15
4800/4800 [==============================] - 72s 15ms/step - loss: 1.0547 - acc: 0.5450 - val_loss: 1.0610 - val_acc: 0.5466
Epoch 7/15
4800/4800 [==============================] - 70s 15ms/step - loss: 1.0356 - acc: 0.5571 - val_loss: 1.0153 - val_acc: 0.5691


In [5]:
import glob

In [23]:
path = '/Users/qianzhuang/Academic/Upenn/Capstone/data/data_csv/pic' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df_pic = pd.read_csv(filename, index_col=None, header=0)
    li.append(df_pic)

frame = pd.concat(li, axis=0, ignore_index=True)

In [24]:
frame.head()

,Unnamed: 0,CityID,pic
0,0,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com..."
1,1,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com..."
2,2,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com..."
3,3,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com..."
4,4,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com..."


In [25]:
len(frame)

10098

In [28]:
pic_address_list = []

In [29]:
def pic2url(pic):
    return pic.split('"')[1]

In [30]:
frame['url'] = frame.pic.apply(pic2url) 

In [31]:
frame.head()

,Unnamed: 0,CityID,pic,url
0,0,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com...",https://dynamic-media-cdn.tripadvisor.com/medi...
1,1,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com...",https://dynamic-media-cdn.tripadvisor.com/medi...
2,2,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com...",https://dynamic-media-cdn.tripadvisor.com/medi...
3,3,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com...",https://dynamic-media-cdn.tripadvisor.com/medi...
4,4,g60745,"url(""https://dynamic-media-cdn.tripadvisor.com...",https://dynamic-media-cdn.tripadvisor.com/medi...


In [86]:
save_path = "/Users/qianzhuang/Academic/Upenn/Capstone/model/Script_pic/"

In [37]:
for i in range(0,len(frame)):
    url = frame.url[i]
    try:
        response = requests.get(url)
    except:
        continue
    img = Image.open(BytesIO(response.content))
    out = img.resize((64, 64))
    resize_path = save_path+frame.CityID[i]
    if not os.path.exists(resize_path):
        os.makedirs(resize_path)
    try:
        out.save(resize_path+"/"+str(i)+".jpg")
    except:
        os.remove(resize_path+"/"+str(i)+".jpg")

In [87]:
files_citys = []
for r, d, f in os.walk(save_path):
    for file in f:
        if '.jpg' in file:
            files_citys.append(os.path.join(r, file))

In [88]:
df_city = pd.DataFrame(files_citys)
df_city = df_city.rename({0:"path"},axis = 1)

In [90]:
df_city.head()

,path
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...


In [91]:
def getCityID(path):
    return path.split("/")[-2];

In [92]:
df_city['CityID'] = df_city.path.apply(getCityID)

In [93]:
df_city.head()

,path,CityID
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391


In [94]:
df_city['RGB'] = df_city.path.apply(creatRGB)

In [95]:
df_city.head()

,path,CityID,RGB
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[89, 139, 190], [92, 143, 196], [97, 148, 20..."
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[32, 22, 21], [10, 0, 0], [20, 12, 10], [31,..."
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[45, 7, 20], [38, 0, 9], [55, 12, 21], [61, ..."
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[36, 53, 109], [36, 53, 109], [37, 54, 110],..."
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[107, 90, 72], [104, 88, 72], [99, 86, 70], ..."


In [96]:
X_city= np.array(df_city.RGB.to_list())/255


In [97]:
result_label = model.predict_classes(X_city)

In [98]:
df_city['Predict'] = result_label

In [99]:
df_city.head()

,path,CityID,RGB,Predict
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[89, 139, 190], [92, 143, 196], [97, 148, 20...",3
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[32, 22, 21], [10, 0, 0], [20, 12, 10], [31,...",2
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[45, 7, 20], [38, 0, 9], [55, 12, 21], [61, ...",2
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[36, 53, 109], [36, 53, 109], [37, 54, 110],...",3
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[107, 90, 72], [104, 88, 72], [99, 86, 70], ...",3


In [104]:
list_label = ['architecure', 'art&culture', 'food&drinks', 'travel&adventure']

In [101]:
def getLabel(num):
    return list_label[num]

In [105]:
df_city['label'] = df_city.Predict.apply(getLabel)

In [106]:
df_city.head()

,path,CityID,RGB,Predict,label
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[89, 139, 190], [92, 143, 196], [97, 148, 20...",3,travel&adventure
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[32, 22, 21], [10, 0, 0], [20, 12, 10], [31,...",2,food&drinks
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[45, 7, 20], [38, 0, 9], [55, 12, 21], [61, ...",2,food&drinks
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[36, 53, 109], [36, 53, 109], [37, 54, 110],...",3,travel&adventure
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,"[[[107, 90, 72], [104, 88, 72], [99, 86, 70], ...",3,travel&adventure


In [109]:
list_cityID = df_city.CityID.unique()

In [114]:
list_cityID

array(['g42391', 'g28970', 'g45926', 'g60898', 'g54184', 'g46874',
       'g49463', 'g60745', 'g44926', 'g54171', 'g32999', 'g49049',
       'g54760', 'g57324', 'g37835', 'g60958', 'g33804', 'g49709',
       'g30196', 'g58653', 'g51947', 'g60893', 'g29494', 'g60805',
       'g147415', 'g60747', 'g44526', 'g34009', 'g60982', 'g43501',
       'g60922', 'g29786', 'g37209', 'g34675', 'g60954', 'g55229',
       'g50226', 'g51560', 'g33388', 'g52787', 'g29212', 'g39426',
       'g40024'], dtype=object)

In [120]:
cityUrl = ['https://www.tripadvisor.com/Tourism-g32999-Sacramento_California-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g33388-Denver_Colorado-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g33804-Hartford_Connecticut-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g34009-Dover_Delaware-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g28970-Washington_DC_District_of_Columbia-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g34675-Tallahassee_Florida-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60898-Atlanta_Georgia-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60982-Honolulu_Oahu_Hawaii-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g44926-Springfield_Missouri-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g37209-Indianapolis_Indiana-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g37835-Des_Moines_Iowa-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60747-Topeka_Kansas-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g39426-Frankfort_Kentucky-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g40024-Baton_Rouge_Louisiana-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g29212-Augusta_Georgia-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g29494-Annapolis_Maryland-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60745-Boston_Massachusetts-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g42391-Lansing_Ingham_County_Michigan-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g43501-Saint_Paul_Minnesota-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60805-Jacksonville_Florida-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g44526-Jefferson_City_Missouri-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g51947-Lincoln_City_Oregon-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g45926-Carson_City_Nevada-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g49049-Concord_North_Carolina-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g46874-Trenton_New_Jersey-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60958-Santa_Fe_New_Mexico-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g29786-Albany_New_York-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g49463-Raleigh_North_Carolina-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g49709-Bismarck_North_Dakota-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g50226-Columbus_Ohio-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g51560-Oklahoma_City_Oklahoma-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60954-Salem_Massachusetts-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g52787-Harrisburg_Pennsylvania-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g147415-New_Providence_Island_Bahamas-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g54184-Columbia_South_Carolina-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g54760-Pierre_South_Dakota-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g55229-Nashville_Davidson_County_Tennessee-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g30196-Austin_Texas-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60922-Salt_Lake_City_Utah-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g57324-Montpelier_Vermont-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g60893-Richmond_Virginia-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g58653-Olympia_Washington-Vacations.html',
 'https://www.tripadvisor.com/Tourism-g54171-Charleston_South_Carolina-Vacations.html']

In [116]:
List_cityUrlID = []
List_cityUrlName = []

In [117]:
for i in range(0,len(cityUrl)):
    List_cityUrlID.append(cityUrl[i].split('-')[1])
    List_cityUrlName.append(cityUrl[i].split('-')[-2])

In [118]:
List_cityUrlID

['g32999',
 'g33388',
 'g33804',
 'g34009',
 'g28970',
 'g34675',
 'g60898',
 'g60982',
 'g44926',
 'g37209',
 'g37835',
 'g60747',
 'g39426',
 'g40024',
 'g29212',
 'g29494',
 'g60745',
 'g42391',
 'g43501',
 'g60805',
 'g44526',
 'g51947',
 'g45926',
 'g49049',
 'g46874',
 'g60958',
 'g29786',
 'g49463',
 'g49709',
 'g50226',
 'g51560',
 'g60954',
 'g52787',
 'g147415',
 'g54184',
 'g54760',
 'g55229',
 'g30196',
 'g60922',
 'g57324',
 'g60893',
 'g58653',
 'g54171']

In [119]:
List_cityUrlName

['Sacramento_California',
 'Denver_Colorado',
 'Hartford_Connecticut',
 'Dover_Delaware',
 'Washington_DC_District_of_Columbia',
 'Tallahassee_Florida',
 'Atlanta_Georgia',
 'Honolulu_Oahu_Hawaii',
 'Springfield_Missouri',
 'Indianapolis_Indiana',
 'Des_Moines_Iowa',
 'Topeka_Kansas',
 'Frankfort_Kentucky',
 'Baton_Rouge_Louisiana',
 'Augusta_Georgia',
 'Annapolis_Maryland',
 'Boston_Massachusetts',
 'Lansing_Ingham_County_Michigan',
 'Saint_Paul_Minnesota',
 'Jacksonville_Florida',
 'Jefferson_City_Missouri',
 'Lincoln_City_Oregon',
 'Carson_City_Nevada',
 'Concord_North_Carolina',
 'Trenton_New_Jersey',
 'Santa_Fe_New_Mexico',
 'Albany_New_York',
 'Raleigh_North_Carolina',
 'Bismarck_North_Dakota',
 'Columbus_Ohio',
 'Oklahoma_City_Oklahoma',
 'Salem_Massachusetts',
 'Harrisburg_Pennsylvania',
 'New_Providence_Island_Bahamas',
 'Columbia_South_Carolina',
 'Pierre_South_Dakota',
 'Nashville_Davidson_County_Tennessee',
 'Austin_Texas',
 'Salt_Lake_City_Utah',
 'Montpelier_Vermont',
 'R

In [121]:
len(list_cityID)

43

In [122]:
len(List_cityUrlID)

43

In [ ]:
df_city = df_city.drop(['RGB'], axis=1)

In [124]:
def getCityName(num):
    return List_cityUrlName[List_cityUrlID.index(num)]

In [132]:
df_city['Full_name'] = df_city.CityID.apply(getCityName)

In [133]:
df_city.head()

,path,CityID,Predict,label,Full_name
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,2,food&drinks,Lansing_Ingham_County_Michigan
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,2,food&drinks,Lansing_Ingham_County_Michigan
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan


In [134]:
def getState(string):
    return string.split('_')[-1]
def getCity(string):
    return string.split('_')[0]

In [136]:
df_city['City'] = df_city.Full_name.apply(getCity)
df_city['State'] = df_city.Full_name.apply(getState)

In [137]:
df_city.head()

,path,CityID,Predict,label,Full_name,City,State
0,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan,Lansing,Michigan
1,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,2,food&drinks,Lansing_Ingham_County_Michigan,Lansing,Michigan
2,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,2,food&drinks,Lansing_Ingham_County_Michigan,Lansing,Michigan
3,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan,Lansing,Michigan
4,/Users/qianzhuang/Academic/Upenn/Capstone/mode...,g42391,3,travel&adventure,Lansing_Ingham_County_Michigan,Lansing,Michigan


In [138]:
df_city.to_csv('./df_city.csv')